# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [2]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [3]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

AttributeError: module 'gym.envs.box2d' has no attribute 'LunarLander'

In [3]:
print(env.observation_space, env.action_space)
print(env.observation_space.sample())

Box(8,) Discrete(4)
[ 1.4045856  -0.56549156  0.32035747  0.51354957  0.22564158  0.9730187
 -0.81774336 -1.2497183 ]


Before running the next code cell, familiarize yourself with the code in **Step 2** and **Step 3** of this notebook, along with the code in `dqn_agent.py` and `model.py`.  Once you have an understanding of how the different files work together, 
- Neural network architecture is defined in `model.py` that maps states to action values.
- The `dqn_agent.py` defines the working of agent through it's interaction with the environment and neural net architecture to optimize q-value function approximator to get better action-value function with each episode. Both the target and local networks are trained on minibatches from memory(Replay buffer)

The below code runs the agent in environment using random actions. Since we don't call the `agent.step()`(update qnet) function here.

In [4]:
from dqn_agent import Agent
import time
agent = Agent(state_size=8, action_size=4, seed=0)
t= 0
# watch an untrained agent running for 1 episode.
state = env.reset()
for j in range(200):
    action = agent.act(state)
    time.sleep(.0125)
    env.render()
    state, reward, done, _ = env.step(action)
    t += reward
    if done:
        print(t)
        break 
print(reward)      
env.close()

-119.12662729928908
-100


### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance! The task is said to be completed, when a score of 200 or above is achieved. I have set this to 240, to get better performance out of it.

In [ ]:
def dqn(n_episodes=2000, max_t=200, eps_start=1.0, eps_end=0.01, eps_decay=0.990):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            #env.render()
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth') 
            break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: -115.93
Episode 200	Average Score: -29.012
Episode 300	Average Score: -2.713
Episode 400	Average Score: 13.07
Episode 500	Average Score: 18.06
Episode 600	Average Score: 28.03
Episode 700	Average Score: 45.86
Episode 800	Average Score: 51.73
Episode 900	Average Score: 67.94
Episode 1000	Average Score: 72.65
Episode 1100	Average Score: 77.82
Episode 1200	Average Score: 84.61
Episode 1300	Average Score: 95.56
Episode 1400	Average Score: 99.00
Episode 1439	Average Score: 99.057

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [5]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(5):
    state = env.reset()
    for j in range(200):
        action = agent.act(state)
        env.render()
        time.sleep(0.0125)
        state, reward, done, _ = env.step(action)
        if done:
            break 
            
env.close()

### 5. Explore

I implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  